In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import unicodedata
import string
import os
from tqdm.auto import tqdm
import io
import glob
import random

In [ ]:
input_size = 28
sequence_length = 28
num_classes = 10
num_layers = 2
hidden_size = 256
learning_rate = 0.001
batch_size = 32
num_of_epochs = 5

# Creating RNN Models

## RNN version 1

In [3]:
class RNNversion1(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNversion1, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x: torch.Tensor):
       initial_hidden_state = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
       x, _ = self.rnn(x, initial_hidden_state)
       x  = x[:, -1, :]
       x = self.fc(x)
       return x

## RNN Version 2

In [80]:
class RNNversion2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNversion2, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x: torch.Tensor):
        ##initialize hidden state
        initial_hidden_state = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
        x, _ = self.rnn(x, initial_hidden_state)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

## LSTM

In [4]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MyModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Sequential(
            # nn.Flatten(start_dim=1),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
    def forward(self, x: torch.Tensor, h0=None, c0=None):
        ##initialize hidden state
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
            c0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [5]:
# cnn_params = [16, 32, "MaxPool"]
cnn_params = [16, 32, 64, "MaxPool"]
class CNN1D(nn.Module):
    def __init__(self, input):
        super(CNN1D, self).__init__()
        self.input = input
        self.conv_blocks = self.create_conv_blocks(cnn_params)
        
    def create_conv_blocks(self, architecture):
        layers = []
        input = self.input

        for x in architecture:
            if type(x) == int:
                output = x
                layers += [nn.Conv1d(input, output, 
                        kernel_size=2, stride=1, padding=0),
                        nn.BatchNorm1d(x),
                        nn.SiLU()]

                input = x  # Update input channels
            elif x == "MaxPool":
                layers += [nn.MaxPool1d(kernel_size=2, stride=1)]

        return nn.Sequential(*layers)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch, features, time_steps) → (batch, channels, time_steps)
        x = self.conv_blocks(x)
        return x  # Output shape: (batch, channels, reduced_time_steps)

class MainModel(nn.Module):
    def __init__(self, input_size=16, hidden_size=100, num_layers=3, output_size=1):
        super(MainModel, self).__init__()
        self.num_layers = num_layers  
        self.hidden_size = hidden_size
        self.cnn = CNN1D(input_size)
        self.lstm = nn.LSTM(input_size=64, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, output_size), 
        )

    def forward(self, x: torch.Tensor, h0=None, c0=None):
        x = self.cnn(x)
        x = x.permute(0, 2, 1)  # (batch, channels, time) → (batch, time, channels)
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
            c0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size)
            # h0 = torch.randn(self.num_layers, x.size(0), self.hidden_size) * 0.02
            # c0 = torch.randn(self.num_layers, x.size(0), self.hidden_size) * 0.02
        x, _ = self.lstm(x, (h0, c0))
        x = x[:, -1, :]
        x = self.fc(x) 
        return x


In [12]:
# Hyperparameters
input_size = 2
hidden_size = 100
num_layers = 20
num_classes = 50
sequence_length = 10
batch_size = 4

# Generate dummy data
# Input tensor of shape (batch_size, sequence_length, input_size)
dummy_input = torch.randn(batch_size, sequence_length, input_size)

# Instantiate the model
# model1 = RNNversion1(input_size, hidden_size, num_layers, num_classes)
# model2 = RNNversion2(input_size, hidden_size, num_layers, num_classes)
model2 = MainModel(input_size, hidden_size, num_layers, num_classes)

# Forward pass with dummy data
# output1 = model1(dummy_input)
output2 = model2(dummy_input)

# print("Input shape:", dummy_input.shape)
# print("Output shape:", output1.shape)
# print("Output:", output1)
print("Output shape:", output2.shape)
print("Output:", output2)

Output shape: torch.Size([4, 50])
Output: tensor([[ 0.0382,  0.0258, -0.0794, -0.0333,  0.0428,  0.0683, -0.0161, -0.1159,
         -0.0077, -0.0243,  0.0260,  0.0108,  0.0755,  0.0415, -0.0002,  0.0283,
         -0.0131,  0.0519, -0.0693,  0.0714, -0.0221, -0.0531, -0.0494, -0.1235,
         -0.0257,  0.0996, -0.1192,  0.0287,  0.0228, -0.0788,  0.0842,  0.0196,
         -0.0761, -0.0274,  0.0693,  0.0882, -0.0387, -0.0659,  0.0592, -0.0122,
         -0.0551,  0.0001,  0.0027, -0.0318,  0.0574,  0.0788, -0.0406,  0.0152,
         -0.0415,  0.1062],
        [ 0.0382,  0.0258, -0.0794, -0.0333,  0.0428,  0.0683, -0.0161, -0.1159,
         -0.0077, -0.0243,  0.0260,  0.0108,  0.0755,  0.0415, -0.0002,  0.0283,
         -0.0131,  0.0519, -0.0693,  0.0714, -0.0221, -0.0531, -0.0494, -0.1235,
         -0.0257,  0.0996, -0.1192,  0.0287,  0.0228, -0.0788,  0.0842,  0.0196,
         -0.0761, -0.0274,  0.0693,  0.0882, -0.0387, -0.0659,  0.0592, -0.0122,
         -0.0551,  0.0001,  0.0027, -0.